# Introduction to Regression Models — Part 3
---


## 1. Preámbulo/encabezado

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

## 2. Datos

Datos de calibración de un FlexiForce.

In [ ]:
v_prom = np.array([0, 0.21, 0.48, 1.12, 1.27, 1.31, 1.43, 1.7, 1.92, 2.24, 2.3, 2.41, 2.53])  # [voltios]
masa   = np.array([10, 20, 50, 100, 120, 150, 200, 250, 300, 350, 400, 450, 500])             # [gramos]

## 3. Entrenamiento del modelo de regresión

### 3.1 Selección del orden del modelo
Modifique `orden` a 1, 2, 3, ... y ejecute nuevamente el notebook para observar el efecto.

In [ ]:
# Selección del orden del polinomio (equivalente a MATLAB "orden")
orden = 1

# scikit-learn requiere X en formato 2D: (n_muestras, n_características)
X = masa.reshape(-1, 1)
y = v_prom

# Pipeline: características polinómicas + regresión lineal
modelo = make_pipeline(
    PolynomialFeatures(degree=orden, include_bias=False),
    LinearRegression()
)

# Entrenamiento
modelo.fit(X, y)

### 3.2 Parametros del modelo

In [ ]:
# Parámetros del modelo: v(m) = b0 + b1*m + b2*m^2 + ... + b_orden*m^orden
lin = modelo.named_steps["linearregression"]

b0 = lin.intercept_
b = lin.coef_  # [b1, b2, ..., b_orden] porque include_bias=False

print(f"Orden: {orden}")
print(f"b0: {b0:.6f}")
for i, bi in enumerate(b, start=1):
    print(f"b{i}: {bi:.6e}")

### 3.3 Evaluación/validación del modelo
Metricas: RMSE y $R^2$

Sea un conjunto de $n$ datos $\{(x_i, y_i)\}_{i=1}^{n}$, y $\hat{y}_i$ la predicción del modelo para $x_i$.

**1) RMSE (Root Mean Squared Error)**  
Mide el error promedio en las mismas unidades de $y$ (por ejemplo, voltios):

$$
\mathrm{RMSE}=\sqrt{\frac{1}{n}\sum_{i=1}^{n}\left(y_i-\hat{y}_i\right)^2}
$$

**2) $R^2$ (Coeficiente de determinación)**  
Indica qué fracción de la variabilidad de $y$ explica el modelo:

$$
R^2 = 1-\frac{\sum_{i=1}^{n}\left(y_i-\hat{y}_i\right)^2}{\sum_{i=1}^{n}\left(y_i-\bar{y}\right)^2}
$$

donde $\bar{y}$ es el promedio de los valores observados:

$$
\bar{y}=\frac{1}{n}\sum_{i=1}^{n}y_i
$$

**Interpretación rápida:**  
- RMSE más bajo $\rightarrow$ mejor ajuste.  
- $R^2$ más cercano a 1 $\rightarrow$ mejor ajuste (en general).


In [ ]:
# Desempeño del ajuste sobre los puntos de entrenamiento
y_hat = modelo.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_hat))
r2 = r2_score(y, y_hat)

# RMSE: error cuadrático medio (en voltios).
# R^2: coeficiente de determinación (0 a 1).
print(f"RMSE (entrenamiento): {rmse:.6f}")
print(f"R^2  (entrenamiento): {r2:.6f}")

### 3.4 Predicción del modelo

In [ ]:
# Respuesta del modelo
x = np.linspace(masa.min(), masa.max(), 20).reshape(-1, 1)
v_regresion = modelo.predict(x)

### 4. Visualización de resultados

Gráfica de los datos de calibración y la curva de regresión.

In [ ]:
plt.figure()
plt.plot(masa, v_prom, 'o', label='Datos de calibración')
plt.grid(True)

# Curva de regresión
plt.plot(x.ravel(), v_regresion, '-', label='Regresión')

plt.title('Datos de calibración y curva de regresión')
plt.xlabel('masa [gramos]')
plt.ylabel('voltaje [voltios]')
plt.legend()
plt.show()

## 5. Discusión

Ejecute nuevamente el script variando el **orden del polinomio** (*orden = 1, 2, 3, ...*). Para cada caso, observe la curva obtenida y compare el desempeño del ajuste (por ejemplo, usando **RMSE** y **(R^2)**).

* **¿Qué orden y qué coeficientes** producen el ajuste más adecuado para los datos de calibración?
  Justifique su elección considerando tanto las métricas numéricas como el comportamiento visual de la curva (suavidad, oscilaciones y coherencia con la tendencia de los datos).
